In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Home.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"



In [3]:


from django.shortcuts import get_object_or_404
from django.db import models

# Gerekli modellerin importları
from analiztest.models import Match, Period, Group, StatisticsItem, PercentValue

In [14]:


def process_match_data(match_data):
 
    # 2. Her periyot için veri ekle
    for period_data in match_data['data']['statistics']:
        period_instance = Period.objects.create(
            match=match_data["match_id"],
            period_name=period_data['period']
        )

        # 3. Her grubu ekle
        for group_data in period_data['groups']:
            group_instance = Group.objects.create(
                period=period_instance,
                group_name=group_data['groupName']
            )

            # 4. Her istatistiği ekle
            for stat_item in group_data['statisticsItems']:
                statistics_item_instance = StatisticsItem.objects.create(
                    group=group_instance,
                    name=stat_item['name'],
                    home_value=stat_item.get('homeValue'),
                    away_value=stat_item.get('awayValue'),
                    compare_code=stat_item.get('compareCode'),
                    statistics_type=stat_item.get('statisticsType'),
                    value_type=stat_item.get('valueType'),
                    render_type=stat_item.get('renderType'),
                    key=stat_item['key']
                )

                # 5. Eğer yüzdeli veri varsa, PercentValue ile sakla
                if 'homeTotal' in stat_item and 'awayTotal' in stat_item:
                    PercentValue.objects.create(
                        statistics_item=statistics_item_instance,
                        home_total=stat_item['homeTotal'],
                        away_total=stat_item['awayTotal'],
                        home_percentage=stat_item.get('homeValue'),
                        away_percentage=stat_item.get('awayValue')
                    )


In [10]:
import ssl
import http.client
import json
import pandas as pd

class SofaScoreAPI:
    def __init__(self, match_id):
        self.match_id = match_id
        self.base_url = 'www.sofascore.com'

    def _make_request(self, endpoint):
        """
        Belirtilen endpoint'e HTTP isteği yapar ve yanıtı döner.
        """
        context = ssl._create_unverified_context()
        con = http.client.HTTPSConnection(self.base_url, context=context)
        try:
            con.request("GET", endpoint)
            response = con.getresponse()
            data = response.read()
            data = json.loads(data)
            return {"match_id":self.match_id,"response_code": response.status, "data": data}
        except Exception as e:
            return {"match_id":self.match_id,"response_code": response.status , "data": None }

    def get_lineups(self):
        """
        Kadro bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/lineups'
        return self._make_request(endpoint)

    def get_incidents(self):
        """
        Maç olaylarını döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/incidents'
        return self._make_request(endpoint)

    def get_graph(self):
        """
        Maç baskı grafiğini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/graph'
        return self._make_request(endpoint)

    def get_match_info(self):
        """
        Maç bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}'
        return self._make_request(endpoint)

    def get_odds(self):
        """
        Maç oran bilgilerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/odds/672/featured'
        return self._make_request(endpoint)


    def get_statistics(self):
        """
        Maç istatistiklerini döndürür.
        """
        endpoint = f'/api/v1/event/{self.match_id}/statistics'
        return self._make_request(endpoint)

    def fetch_all_data(self):
        """
        Tüm API endpoint'lerinden verileri toplar ve bir dict olarak döner.
        """
        return {
            "match_id": self.match_id,
            "lineups": self.get_lineups(),
            "incidents": self.get_incidents(),
            "graph": self.get_graph(),
            "match_info": self.get_match_info(),
            "odds": self.get_odds(),
            "pregame_form": self.get_pregame_form(),
            "statistics": self.get_statistics(),
        }

# Örnek kullanım:
match_id = 11450061
sofa_api = SofaScoreAPI(match_id)
result_all = sofa_api.get_statistics()
result_all


{'match_id': 11450061,
 'response_code': 200,
 'data': {'statistics': [{'period': 'ALL',
    'groups': [{'groupName': 'Match overview',
      'statisticsItems': [{'name': 'Ball possession',
        'home': '60%',
        'away': '40%',
        'compareCode': 1,
        'statisticsType': 'positive',
        'valueType': 'event',
        'homeValue': 60,
        'awayValue': 40,
        'renderType': 2,
        'key': 'ballPossession'},
       {'name': 'Expected goals',
        'home': '2.29',
        'away': '0.51',
        'compareCode': 1,
        'statisticsType': 'positive',
        'valueType': 'event',
        'homeValue': 2.29,
        'awayValue': 0.51,
        'renderType': 1,
        'key': 'expectedGoals'},
       {'name': 'Big chances',
        'home': '3',
        'away': '0',
        'compareCode': 1,
        'statisticsType': 'positive',
        'valueType': 'event',
        'homeValue': 3,
        'awayValue': 0,
        'renderType': 1,
        'key': 'bigChanceCreated'

In [15]:
process_match_data(result_all)

ValueError: Cannot assign "11450061": "Period.match" must be a "Match" instance.